In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 4-5. ユーザ定義型

### 4-5-1. ユーザによる型の定義

#### コード4-12. 型定義の例 (1)（再掲）

In [2]:
struct Decimal <: Real
    value::BigInt
    point::Int
end

#### コード4-46. 型定義の例 (3)：抽象型の定義例

In [3]:
abstract type AbstractQDInfo end

In [4]:
abstract type AbstractQDError <: AbstractQDInfo end

In [5]:
abstract type AbstractData{T} end

In [6]:
abstract type AbstractPoint{T<:Real} end

#### コード4-47. 型定義の例 (4)：構造体の定義例

In [7]:
struct Decimal <: Real
    value::BigInt
    point::Int
end

In [8]:
struct Point2D{T} <: AbstractPoint{T}
    x::T
    y::T
end

In [9]:
struct MyIntegerArray{T<:Integer, N} <: AbstractArray{T, N}
    inner::Array{T, N}
end

In [10]:
struct MySingleton end

#### コード4-48. 型定義の例 (5)：可変構造体の定義例

In [11]:
# このまま実行すると後ほど「invalid redefinition of constant Future」というエラーが出てしまうのでコメントアウト
#=
mutable struct Future
    value
    done::Bool
end
=#

In [12]:
mutable struct MPoint2D{T} <: AbstractPoint{T}
    x::T
    y::T
end

#### コード4-49. 構造体と可変構造体の違い確認例

In [13]:
mpt = MPoint2D(0, 0)

MPoint2D{Int64}(0, 0)

In [14]:
mpt.x = 1;
mpt.y = 2;
mpt

MPoint2D{Int64}(1, 2)

In [15]:
pt = Point2D(0, 0)

Point2D{Int64}(0, 0)

In [16]:
pt.x = 1

LoadError: setfield!: immutable struct of type Point2D cannot be changed

#### コード4-50. 型定義の例 (6)：プリミティブ型の定義例

In [17]:
primitive type Int24 <: Signed 24 end

In [18]:
primitive type Address{T} 64 end

### 4-5-2. コンストラクタ定義

#### コード4-51. `Point2D{T}` 型の定義例

In [19]:
struct Point2D{T} <: AbstractPoint{T}
    x::T
    y::T
end

In [20]:
pt = Point2D(1, 2)

Point2D{Int64}(1, 2)

In [21]:
pt.x

1

In [22]:
pt.y

2

#### コード4-52. `MPoint2D{T}` 型とその外部コンストラクタの定義例

In [23]:
mutable struct MPoint2D{T} <: AbstractPoint{T}
    x::T
    y::T
end

In [24]:
function MPoint2D(v)
    MPoint2D(v, v)
end

MPoint2D

In [25]:
function MPoint2D()
    MPoint2D(0.0)
end

MPoint2D

In [26]:
function MPoint2D{T}() where T<:Real
    MPoint2D(zero(T))
end

#### コード4-53. `MPoint2D{T}` 型のインスタンス生成例

In [27]:
MPoint2D(1, 2)

MPoint2D{Int64}(1, 2)

In [28]:
MPoint2D(1)

MPoint2D{Int64}(1, 1)

In [29]:
MPoint2D()

MPoint2D{Float64}(0.0, 0.0)

In [30]:
MPoint2D{Int}()

MPoint2D{Int64}(0, 0)

#### コード4-54. `Future` 型とその内部コンストラクタの定義例

In [31]:
mutable struct Future
    value
    done::Bool

    function Future()
        object = new()
        object.done = false
        object
    end
    function Future(value)
        new(value, true)
    end
end

#### コード4-55. `Future` 型のインスタンス生成例

In [32]:
f0 = Future()

Future(#undef, false)

In [33]:
f1 = Future("Concrete Value")

Future("Concrete Value", true)

In [34]:
f2 = Future("Concrete Value", true)

LoadError: MethodError: no method matching Future(::String, ::Bool)
[0mClosest candidates are:
[0m  Future(::Any) at In[31]:10

#### コード4-56. 加工・変換のための外部コンストラクタの例 (1)

In [35]:
function Point2D(t::NTuple{2, T}) where T<:Real
    Point2D(t...)
end

Point2D

In [36]:
function MPoint2D(t::NTuple{2, T}) where T<:Real
    MPoint2D(t...)
end

MPoint2D

In [37]:
function Point2D(mpt::MPoint2D)
    Point2D(mpt.x, mpt.y)
end

Point2D

In [38]:
function MPoint2D(pt::Point2D)
    MPoint2D(pt.x, pt.y)
end

MPoint2D

#### コード4-57. 加工・変換のための外部コンストラクタの例 (2)

In [39]:
t1 = (1, 2)

(1, 2)

In [40]:
pt1 = Point2D(t1)

Point2D{Int64}(1, 2)

In [41]:
mpt1 = MPoint2D(t1)

MPoint2D{Int64}(1, 2)

In [42]:
Point2D(mpt1)

Point2D{Int64}(1, 2)

In [43]:
MPoint2D(pt1)

MPoint2D{Int64}(1, 2)